In [6]:
import json
import re
import pandas as pd
import numpy as np

In [8]:
import json
import re
import pandas as pd

# Load the JSON file
file_path = "ml_project_train_and_evaluation.json"  # Replace with your file path
with open(file_path, "r") as file:
    json_data = json.load(file)

# Extract all outputs and their corresponding sources
outputs_with_sources = []
for cell in json_data["cells"]:
    if cell["cell_type"] == "code" and "outputs" in cell:
        source = "\n".join(cell["source"])
        for output in cell["outputs"]:
            if "text" in output:
                outputs_with_sources.append((source, "\n".join(output["text"])))

# Initialize a results list
results_list = []

# Function to extract pipeline prefix
def extract_source(command):
    match = re.match(r"evaluate_model\.(\w+)_model\(\)", command)
    return match.group(1) if match else "Unknown"

df_summary = pd.DataFrame(outputs_with_sources)

# Display the DataFrame
print(df_summary)

# Save to CSV
output_csv_path = "parsed_metrics_with_source.csv"
df_summary.to_csv(output_csv_path, index=False)
print(f"Results saved to: {output_csv_path}")

                                                    0  \
0   # 1. Logistic Regression on W100_O25_Features\...   
1   # 2. Decision Tree Classifier on W100_O25_Feat...   
2   # 3. Random Forest Classifier on W100_O25_Feat...   
3   # 4. Gaussian Naive Bayes on W100_O25_Features...   
4   # 5. Support Vector Classifier on W100_O25_Fea...   
5   # 6. K-Nearest Neighbors on W100_O25_Features\...   
6   # 7. AdaBoost Classifier on W100_O25_Features\...   
7   # 8. Gradient Boost on W100_O25_Features\n\nev...   
8   # 9. XGBoost Classifier on W100_O25_Features\n...   
9   # 1. Logistic Regression on W100_O50_Features\...   
10  # 2. Decision Tree Classifier on W100_O50_Feat...   
11  # 4. Gaussian Naive Bayes on W100_O50_Features...   
12  # 5. Support Vector Classifier on W100_O50_Fea...   
13  # 6. K-Nearest Neighbors on W100_O50_Features\...   
14  # 7. AdaBoost Classifier on W100_O50_Features\...   
15  # 9. XGBoost Classifier on W100_O50_Features\n...   
16  # 1. Logistic Regression on

In [9]:
source_file = pd.read_csv("parsed_metrics_with_source.csv")
source_file.head()


,0,1
0,# 1. Logistic Regression on W100_O25_Features\...,#------------------- #1. Logistic Regression M...
1,# 2. Decision Tree Classifier on W100_O25_Feat...,#-------------------- #2. Decission Tree Class...
2,# 3. Random Forest Classifier on W100_O25_Feat...,#-------------------- #3. Random Forest Classi...
3,# 4. Gaussian Naive Bayes on W100_O25_Features...,#-------------------- #4. Gaussian Naive Bias ...
4,# 5. Support Vector Classifier on W100_O25_Fea...,#-------------------- #5. Support Vector Class...


In [11]:
results_list = []

def parse_metrics_flexible(source, text):
    try:
        # Define improved regex patterns
        model_pattern = r"#-+\s*#\d+\.\s*(.*?)\s*-+#"
        accuracy_pattern = r"accuracy\s+([\d.]+)"
        weighted_avg_pattern = r"weighted avg\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)"

        if 'ann' in source:
            kfold_pattern = r"K-fold cross-validation scores:.*?Average score:\s*([\d.]+)"
            stratified_pattern = r"Stratified cross-validation scores:.*?Average score:\s*([\d.]+)"
        else:
            kfold_pattern = r"K-fold cross validaiton scores:\s*\[([\d.\s]+)\]"
            stratified_pattern = r"Straified cross validation scores:\s*\[([\d.\s]+)\]"

        # Extract metrics
        model = re.search(model_pattern, text).group(1).strip()
        accuracy_match = re.search(accuracy_pattern, text)
        match = re.search(weighted_avg_pattern, text)
        kfold_match = re.search(kfold_pattern, text)
        stratified_match = re.search(stratified_pattern, text)

        accuracy = float(accuracy_match.group(1)) if accuracy_match else None

        if match:
            precision = float(match.group(1))
            recall = float(match.group(2))
            f1_score = float(match.group(3))
        else:
            print("Weighted avg row not found.")

        if 'ann' in source:
            kfold_match = re.search(kfold_pattern, text, re.DOTALL)
            kfold_avg = float(kfold_match.group(1))

            stratified_match = re.search(stratified_pattern, text, re.DOTALL)
            stratified_avg = float(stratified_match.group(1))

        else:
            kfold_scores = [float(x) for x in kfold_match.group(1).split()]
            kfold_avg = sum(kfold_scores) / len(kfold_scores)
            
            stratified_scores = [float(x) for x in stratified_match.group(1).split()]
            stratified_avg = sum(stratified_scores) / len(stratified_scores)

        # Append to results list
        results_list.append({
            "Model": model,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1_score,
            "k-fold score": kfold_avg,
            "stratified score": stratified_avg,
            "Source": extract_source(source)
        })
    except Exception as e:
        print(f"Error processing source '{source}': {e}")

# Parse all rows with improved regex

for row in source_file.itertuples(index=False):
    parse_metrics_flexible(row[0], row[1])

# Convert parsed results to a DataFrame
df_parsed_metrics = pd.DataFrame(results_list)
print(df_parsed_metrics.head())

# for i in range(5):
#     print(outputs_with_sources[i][1])

df_parsed_metrics.to_csv("evaluation_metrics.csv")

Error processing source '# 1. Logistic Regression on W100_O25_Features

evaluate_model.logistic_regression_model()': 'NoneType' object has no attribute 'group'
Error processing source '# 2. Decision Tree Classifier on W100_O25_Features

evaluate_model.decission_tree_classifier_model()': 'NoneType' object has no attribute 'group'
Error processing source '# 3. Random Forest Classifier on W100_O25_Features

evaluate_model.random_forest_classifier_model()': 'NoneType' object has no attribute 'group'
Error processing source '# 4. Gaussian Naive Bayes on W100_O25_Features

evaluate_model.gaussian_naive_bias_classifier_model()': 'NoneType' object has no attribute 'group'
Error processing source '# 5. Support Vector Classifier on W100_O25_Features

evaluate_model.support_vector_classifier_model()': 'NoneType' object has no attribute 'group'
Error processing source '# 6. K-Nearest Neighbors on W100_O25_Features

evaluate_model.knn_classifier_model()': 'NoneType' object has no attribute 'group'
